In [1]:
import numpy as np
from datetime import datetime

In [2]:
def BB(w0, wt, t, m):
    z = np.random.randn(2**m) # z.len : 8
    h = 2**m # h = 8
    jmax = 1
    w = np.zeros(2**m+1) # w.len : 9
    w[0] = w0
    w[-1] = wt
    tt = np.linspace(0,t,2**m+1) # [0, 1, 2, 3, 4, 5, 6, 7 , 8]
    for k in range(m):
        imin = int(h/2) # imin : 4 -> 2 -> 1
        i = imin
        l = 0
        r = h # r : 8 -> 4 -> 2
        for j in range(jmax):
            a = ((tt[r]-tt[i])*w[l]+(tt[i]-tt[l])*w[r])/(tt[r]-tt[l])
            b = np.sqrt((tt[i]-tt[l])*(tt[r]-tt[i])/(tt[r]-tt[l]))
            w[i] = a+b*z[i-1]
            i = i+h # i : 4 // 2 -> 6 // 1 -> 3 -> 5 -> 7
            l = l+h # l : 0 // 0 -> 4 // 0 -> 2 -> 4 -> 6
            r = r+h # r : 8 // 4 -> 8 // 2 -> 4 -> 6 -> 8
        jmax = 2*jmax # jmax : 1 -> 2 -> 4
        h = imin # h : 8 -> 4 -> 2
    return w
W=BB(0,8,1,3)
W

array([0.        , 0.80634187, 2.45511322, 3.46938095, 4.20187212,
       5.12277806, 6.21201638, 7.04102857, 8.        ])

### Use Brownian bridge

In [240]:
def ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m):
    
    prob = np.zeros(7)
    values = np.zeros(M)
    for i in range(M):
        z1 = np.random.randn(4)
        z2 = np.random.randn(4)
        w1_rw = np.zeros(5)
        w2_rw = np.zeros(5)
        s1_rw = np.zeros(5)
        s2_rw = np.zeros(5)
        s1_rw[0] = 100
        s2_rw[0] = 100
        t_rw = 1/2
        
        for j in range(4):

            w1_rw[j+1] = w1_rw[j] + np.sqrt(t_rw)*z1[j]
            w2_rw[j+1] = w2_rw[j] + np.sqrt(t_rw)*z2[j]

            s1_rw[j+1] = s1_rw[0]*np.exp((r-q1-sigma1**2/2)*t_rw*(j+1) + sigma1*w1_rw[j+1])
            s2_rw[j+1] = s2_rw[0]*np.exp((r-q2-sigma2**2/2)*t_rw*(j+1) + sigma2*(rho*w1_rw[j+1]+np.sqrt(1-rho**2)*w2_rw[j+1]))

        # 1번 필터링 (0.5년 후 종가가 둘 다 85% 이상)
        if (s1_rw[1]>=85) & (s2_rw[1]>=85):
            val = 100 * (1+0.0625)
            prob[0]+=1
            values[i] = val * np.exp(-r * 0.5)

        # 2번 필터링 (1년 후 종가가 둘 다 80% 이상)
        elif (s1_rw[2]>=80) & (s2_rw[2]>=80):
            val = 100 * (1+0.0625*2)
            prob[1]+=1
            values[i] = val * np.exp(-r)

        # 3번 필터링 (1.5년 후 종가가 둘 다 75% 이상)
        elif (s1_rw[3]>=75) & (s2_rw[3]>=75):
            val = 100 * (1+0.0625*3)
            prob[2]+=1
            values[i] = val * np.exp(-r * 1.5)

        # 4번 필터링 (2년 후 종가가 둘 다 70% 이상)
        elif (s1_rw[4]>=70) & (s2_rw[4]>=70):
            val = 100 * (1+0.0625*4)
            prob[3]+=1
            values[i] = val * np.exp(-r * 2)

        # 7번 필터링 (위의 경우에 해당하지 않으면서 barrier hit)
        elif (any(s1_rw <= 60) | any(s2_rw <= 60)) == True:
            val = min(s1_rw[4], s2_rw[4])
            prob[6]+=1
            values[i] = val * np.exp(-r * 2)

        else:
            
            s1_BB = np.zeros(2**m+1)
            s2_BB = np.zeros(2**m+1)
                
            for k in range(4):
                w1_BB = BB(w1_rw[k], w1_rw[k+1], t_rw, m)
                w2_BB = BB(w2_rw[k], w2_rw[k+1], t_rw, m)
                
                t = np.linspace(t_rw*k, t_rw*(k+1), 2**m+1)

                s1_BB = s1_rw[0]*np.exp((r-q1-sigma1**2/2)*t + sigma1*w1_BB)
                s2_BB = s2_rw[0]*np.exp((r-q2-sigma2**2/2)*t + sigma2*(rho*w1_BB+np.sqrt(1-rho**2)*w2_BB)) 
                
                if (any(s1_BB <= 60) | any(s2_BB <= 60)) == True:
                    val = min(s1_rw[4], s2_rw[4])
                    prob[5]+=1
                    values[i] = val * np.exp(-r * 2)
                    break

                else:
                    if k < 3:
                        continue

                    else:
                        val = 100
                        prob[4]+=1
                        values[i] = val * np.exp(-r * 2)
    
    value = values.mean()
    err = values.std() / sqrt(M)
    prob = prob / M
        
    return value, err, prob    

In [126]:
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=3

In [243]:
t1=datetime.now()
value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m)
t2=datetime.now()
print('value = {:.2f}, err = {:.2f}'.format(value,err))
for i in range(7):
    print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i],width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob)))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 93.41, err = 0.27
prob[1] = 55.16%
prob[2] = 12.48%
prob[3] =  6.58%
prob[4] =  4.27%
prob[5] =  0.15%
prob[6] =  1.44%
prob[7] = 19.92%
Total sum of prob = 100%
Total computing time = 1.195996 seconds


In [242]:
import numpy as np
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=7
p=np.zeros(m)
print('      ',end=' ')
for i in range(7):
    print('prob[{:d}]'.format(i+1),end=' ')
print('  computing time')
for i in range(m):
    t1=datetime.now()
    value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,i)
    t2=datetime.now()
    print('n={:3d}'.format(2**i),end='   ')
    for j in range(len(prob)):
        print('{:{width}.2%} '.format(prob[j],width=6),end=' ')
    print(' {:f} seconds'.format((t2-t1).total_seconds()))

       prob[1] prob[2] prob[3] prob[4] prob[5] prob[6] prob[7]   computing time
n=  1   54.52%  12.95%   6.54%   4.15%   1.37%   0.00%  20.47%   0.666257 seconds
n=  2   55.55%  12.18%   6.11%   4.57%   0.98%   0.58%  20.03%   0.730012 seconds
n=  4   54.04%  12.58%   6.46%   4.25%   0.76%   0.79%  21.12%   0.633317 seconds
n=  8   55.05%  12.19%   6.29%   3.82%   0.38%   1.22%  21.05%   0.630837 seconds
n= 16   55.03%  12.45%   6.32%   3.87%   0.25%   1.19%  20.89%   0.644284 seconds
n= 32   54.78%  12.34%   6.26%   4.39%   0.14%   1.24%  20.85%   0.695443 seconds
n= 64   54.70%  12.31%   6.37%   4.03%   0.20%   1.48%  20.91%   0.928889 seconds


In [3]:
import numpy as np
def ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m):

    return value,err,prob
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=3
t1=datetime.now()
value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m)
t2=datetime.now()
print('value = {:.2f}, err = {:.2f}'.format(value,err))
for i in range(7):
    print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i],width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob)))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 93.28, err = 0.27
prob[1] = 55.55%
prob[2] = 12.69%
prob[3] =  6.00%
prob[4] =  4.05%
prob[5] =  0.33%
prob[6] =  1.16%
prob[7] = 20.22%
Total sum of prob = 100%
Total computing time = 0.569032 seconds


In [11]:
import numpy as np
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=7
p=np.zeros(m)
print('      ',end=' ')
for i in range(7):
    print('prob[{:d}]'.format(i+1),end=' ')
print('  computing time')
for i in range(m):
    t1=datetime.now()
    value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,i)
    t2=datetime.now()
    print('n={:3d}'.format(2**i),end='   ')
    for j in range(len(prob)):
        print('{:{width}.2%} '.format(prob[j],width=6),end=' ')
    print(' {:f} seconds'.format((t2-t1).total_seconds()))

       prob[1] prob[2] prob[3] prob[4] prob[5] prob[6] prob[7]   computing time
n=  1   54.60%  12.69%   6.56%   4.32%   1.49%   0.00%  20.34%   0.504029 seconds
n=  2   54.71%  12.48%   6.62%   4.20%   0.85%   0.77%  20.37%   0.527030 seconds
n=  4   55.30%  12.67%   6.53%   4.37%   0.43%   0.85%  19.85%   0.513029 seconds
n=  8   54.87%  11.64%   6.56%   4.08%   0.44%   1.18%  21.23%   0.570033 seconds
n= 16   54.83%  12.52%   6.56%   4.14%   0.31%   1.27%  20.37%   0.621035 seconds
n= 32   55.15%  12.13%   6.60%   4.25%   0.18%   1.31%  20.38%   0.758043 seconds
n= 64   54.80%  12.62%   6.47%   4.26%   0.23%   1.50%  20.12%   1.099063 seconds
